<a href="https://colab.research.google.com/github/talktokorea/DeepLearningBrowser/blob/master/titanic4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

from google.colab import files
from google.colab import drive

files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/*
drive.mount('/content/gdrive')

In [ ]:
train_data = pd.read_csv('./gdrive/MyDrive/Colab Notebooks/data/train.csv')
train_data.info()

In [ ]:
test_data = pd.read_csv('./gdrive/MyDrive/Colab Notebooks/data/test.csv')
test_data.info()

In [ ]:
# 방법1  학습용 X 데이터를 NP 배열 로우=891, 컬럼=(0,7)로 작성
from tensorflow.keras.utils import to_categorical 
#train_data_np = np.zeros([891,8])
train_data_np = np.zeros([train_data.shape[0],8])

sex_num = np.zeros(train_data_np.shape[0])
sex_num[np.array(train_data['Sex']=='female')]=1
train_data_np[:,0:2] = to_categorical(sex_num)  # 남자 0, 여자 1로 구성된 배열을 0 1, 1 0 으로 변경

train_data_np[:,2:5] = to_categorical(train_data['Pclass'].to_numpy()-1)  # 1,2,3을 0,1,2로 변경
train_data_np[:,5] = train_data['Age']/train_data['Age'].max()  # 80  
train_data_np[:,6] = train_data['SibSp']/train_data['SibSp'].max() # 10
train_data_np[:,7] = train_data['Parch']/train_data['Parch'].max()  # 10
train_data_np[np.isnan(train_data_np)] = train_data['Age'].mean()/train_data['Age'].max()  # 30/80
train_data_np_y = to_categorical(train_data['Survived'])

print(train_data_np[0:5,:])

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import optimizers
model = Sequential()
model.add(layers.Dense(units=8, activation='relu'))
model.add(layers.Dense(units=16, activation='relu'))
model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
hist = model.fit(train_data_np,train_data_np_y,epochs= 400, batch_size=128, validation_split=0.3)

In [ ]:
import matplotlib.pyplot as plt

def drawHistory(hist):
  fig = plt.figure(figsize=(15,10)) # 10x15 크기의 figure 생성
  loss_ax = plt.gca()        # figure의 기본 축 가져오기
  acc_ax = loss_ax.twinx()      # 축을 1개 더 추가

  #손실 변화 양상을 그래프로 표시
  loss_ax.plot(hist.history['loss'], 'y', label='train loss')
  loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
  loss_ax.set_xlabel('epoch')
  loss_ax.set_ylabel('loss')
  loss_ax.legend(loc='lower left')

  #정확도 변화 양상을 그래프로 표시
  acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
  acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
  acc_ax.set_ylabel('accuracy')
  acc_ax.legend(loc='upper left')

  plt.show() #그래프를 화면에 그림

drawHistory(hist) #학습 경과를 그래프로 그리는 함수 호출

In [ ]:
# 방법1  테스트용 X 데이터를 NP 배열 로우=, 컬럼=( )로 작성
from tensorflow.keras.utils import to_categorical 
test_data_np = np.zeros([test_data.shape[0],8])

sex_num_test = np.zeros(test_data_np.shape[0])
sex_num_test[np.array(test_data['Sex']=='female')]=1
test_data_np[:,0:2] = to_categorical(sex_num_test)  # 남자 0, 여자 1로 구성된 배열을 0 1, 1 0 으로 변경

test_data_np[:,2:5] = to_categorical(test_data['Pclass'].to_numpy()-1)  # 1,2,3을 0,1,2로 변경
test_data_np[:,5] = test_data['Age']/test_data['Age'].max()  # 80  
test_data_np[:,6] = test_data['SibSp']/test_data['SibSp'].max() # 10
test_data_np[:,7] = test_data['Parch']/test_data['Parch'].max()  # 10
test_data_np[np.isnan(test_data_np)] = test_data['Age'].mean()/test_data['Age'].max()  # 30/80
print(test_data_np[0:5,:])

In [ ]:
o = model.predict(test_data_np)
o = np.argmax(o,-1)  # 두개중 큰값을 이용한 0,1 설정
print(o)

In [ ]:
answer = np.zeros([418,2])
answer[:,0] = test_data['PassengerId']
answer[:,1] = o
df_answer = pd.DataFrame(answer) # 배열 answer를 판다스 라이브러리의 DataFrame 
                                 # 형식으로 변환하고, df_frame 변수에 저장한다.
df_answer.columns =['PassengerID','Survived']       # 각 열의 제목을 지정한다.
df_answer = df_answer.astype('int32')               # 데이터를 정수형으로 바꾼다.
df_answer.to_csv('test_pridict.csv', index = False) # 결과를 csv 파일로 저장한다. 
                                                    # (index는 제외한다)

In [ ]:
!kaggle competitions submit -c titanic -f 'test_pridict.csv' -m "Message"